In [ ]:
%load_ext autoreload
%autoreload 2
%aimport

In [ ]:
import os 
import sys
path = os.path.abspath('../..')
if path not in sys.path: 
    sys.path.insert(0, path)
    print(path)
sys.path[0]

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import torch
import torch.optim as optim

import fitsio 

import json

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('torch version: ', torch.__version__)
print(device)

In [ ]:
from src.data import simulated_datasets_lib
from src import psf_transform_lib
from src.utils import const

# Load 

In [ ]:
# this is the PSF I fitted using ground truth Hubble locations/fluxes. 
init_psf_params = torch.Tensor(np.load('../../data/fitted_powerlaw_psf_params.npy'))
power_law_psf = psf_transform_lib.PowerLawPSF(init_psf_params.to(device))
psf = power_law_psf.forward().detach()

# number of bands. Here, there are two. 
n_bands = psf.shape[0]


In [ ]:
psf.shape

# Plot images

In [ ]:
# data parameters
with open('../../data/default_star_parameters.json', 'r') as fp:
    data_params = json.load(fp)
data_params['max_stars'] = 5
data_params['mean_stars'] = 3
print(data_params)

In [ ]:
# set background 
background = torch.zeros(n_bands, data_params['slen'], data_params['slen'])
background[0] = 686.
background[1] = 1123.

In [ ]:
# draw data 
n_images = 1

simulated_dataset = \
    simulated_datasets_lib.StarsDataset.load_dataset_from_params(psf,
                    data_params,
                    background = background,
                    n_images = n_images,
                    transpose_psf = False, 
                    add_noise = True)

images = simulated_dataset.images.detach()
locs, fluxes, n_stars = simulated_dataset.locs, simulated_dataset.fluxes, simulated_dataset.n_stars

# images is n_images x n_bands x slen x slen
print(images.shape)

# fluxes is n_images x max_stars x n_bands
print(fluxes.shape)

# locs is n_images x max_stars x len((x,y))
print(locs.shape)

#n_stars is (n_images)
print(n_stars.shape)

In [ ]:
is_on_array = const.get_is_on_from_n_sources(n_stars, 5)
is_on_array
# this means that they are only 5 stars in the first image. 

In [ ]:
(2 < n_stars).float()

In [ ]:
n=2
is_on_n = (n < n_stars).float()
locs[:, n, :] * is_on_n.unsqueeze(1)